In [1]:
#selenium 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#추가 조작 라이브러리
import time
import os
from bs4 import BeautifulSoup
from time import sleep

#기타 라이브러리
import pandas as pd


In [2]:
PATH_POST_DATA_CSV = 'data_csv/post.csv'
PATH_POST_DATA = 'data_json/post.json'

URL_BASE = 'https://gall.dcinside.com/board/lists/?id=football_new8&page='

In [3]:
## Functions
def reading_file(file_path):
    encodings = ['cp949', 'utf-8', 'ISO-8859-1', 'cp1252']  # List of encodings to try
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            return df  # or process your df
        except: pass
    return None


In [4]:
import pandas as pd
import json

with open(PATH_POST_DATA, 'r', encoding='utf-8') as f:
    data_post = json.load(f)

def crawling_data():
    contents_box = []
    options = webdriver.ChromeOptions()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    size = 100001
    cnt = 1
    for i in range(1, size)[::100]:
        progress = ((cnt+1) / size) * 100
        if f"{cnt}" in data_post:
            cnt += 1
            continue
        try:
            driver.get(f'{URL_BASE}{i}')
            sleep(3)
        except Exception as e: 
            #예외 발생 시 다시 load
            print(e)
            continue

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        for tr in soup.find_all('tr', {'class':'ub-content us-post'}):
            try:
                YYYY,MM,DD = tr.find('td', {'class':'gall_date'})['title'].split()[0].split('-')
                date = YYYY+MM+DD
                content = tr.find('a').text.strip('\n')
                data_post[f'{cnt}'] = [date, content]
                contents_box.append([date, content])
                with open(PATH_POST_DATA, 'w', encoding='utf-8') as f:
                    json.dump(data_post, f)
                cnt += 1
            except Exception as e:
                print(e)
                pass
    return contents_box

if os.path.exists(PATH_POST_DATA_CSV) == False:
    crawling_data()
    df_post = pd.DataFrame(list(data_post.values()), columns=['DATE', 'CONTENT'])
    df_post.to_csv(PATH_POST_DATA_CSV, index=False)
else:
    df_post = reading_file(PATH_POST_DATA_CSV)

df_post

,DATE,CONTENT
0,20240320,뭐야 내가 글을 잘못 쓴거야?댓글이 병신인거야?
1,20240320,해붕이 요즘 피부상태가 안좋다....
2,20240320,애미창민 은퇴하면 피엘 보지도 않고 조트넘만 보는 손뽕들이
3,20240320,아직도 대가리 안깨진 새끼들은 뭐 ㄹㅇ 앰생이거나 조선족 알바겠지
4,20240320,핑크 ㅂㅈ 유니폼
...,...,...
49890,20221224,[구토멸망] 인성가지고 신두형 뭐라고 못함ㅋㅋ
49891,20221224,우유가 뿜어져나오는 협곡
49892,20221224,결국 호날두 역대 선수 8위 선정
49893,20221224,카카정도면 레전드냐?
